In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from matplotlib import pyplot as plt
import json
import numpy as np
import re
import random
import pandas as pd
from matplotlib import pyplot as plt
import json
import numpy as np
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
from scipy.sparse import coo_matrix, hstack

from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics.cluster import completeness_score

from sklearn.decomposition import TruncatedSVD

from sklearn.cluster import KMeans, MiniBatchKMeans
import datetime
from datetime import timedelta

from sklearn.metrics import silhouette_samples, silhouette_score


In [ ]:
total=pd.read_json("/content/drive/MyDrive/NLP_News/Data/total.json")
total.columns = ['id', 'date','text','username','ALL']
total["ind"]=np.arange(total.shape[0])
total.head()

,id,date,text,username,ALL,ind
0,679451396844990465,2015-12-23 00:00:11,gulf nations demand release of abducted qatari...,ajenews,"[iraq, qataris, gulf nations]",0
1,679451620514611201,2015-12-23 00:01:04,midway through enrollment season for president...,ap,[obama],1
2,679452844307705856,2015-12-23 00:05:56,russia bombs kill syria civilians,bbcworld,"[syria, russia]",2
3,679453367106678785,2015-12-23 00:08:01,six republican candidates expected in fox busi...,cnn,"[fox, republican, six]",3
4,679453408324108288,2015-12-23 00:08:10,video bagpipes amid the bustle in tokyo,bbcworld,[tokyo],4


In [ ]:
m_list=pd.read_csv("/content/drive/MyDrive/NLP_News/Data/till70000.json")
m_list

,Unnamed: 0,week,middle_time,model,tf_idf,start_ind,end_ind
0,0,1.0,2016-04-10 01:01:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,50594,52808
1,1,1.5,2016-04-12 01:01:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,51481,53879
2,2,2.0,2016-04-15 01:02:53,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,52809,55048
3,3,2.5,2016-04-17 01:02:53,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,53883,56061
4,4,3.0,2016-04-20 01:12:09,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,55049,57623
5,5,3.5,2016-04-22 01:12:09,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,56067,58209
6,6,4.0,2016-04-25 01:13:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,57624,59823
7,7,4.5,2016-04-27 01:13:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,58212,60820
8,8,5.0,2016-04-30 01:19:00,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,59824,61945
9,9,5.5,2016-05-02 01:19:00,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,60822,63006


In [ ]:
from datetime import datetime
from datetime import timedelta
for i in range(m_list.shape[0]):
  m_list.middle_time[i]=datetime.strptime(m_list.middle_time[i],'%Y-%m-%d %H:%M:%S')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
col_list = ['title','date','ALL']
truedata=pd.read_csv("/content/drive/MyDrive/NLP_News/Data/True_P.csv",skipinitialspace=True,usecols=col_list)

In [ ]:
truedata.columns = ['text','date','ALL']
truedata

,text,date,ALL
0,obama says must change the way nation manages ...,2016-01-13,['obama']
1,obama says too many americans feel the system ...,2016-01-13,"['americans', 'obama']"
2,biden sanders has struck a chord with voters i...,2016-01-13,"['clinton', 'biden sanders']"
3,south korea president says acquiring nukes wou...,2016-01-13,['south korea']
4,south korea president says china response to n...,2016-01-13,"['north korea', 'south korea', 'china']"
...,...,...,...
20288,trump wants postal service to charge much more...,2017-12-29,"['amazon', 'trump']"
20289,u s military to accept transgender recruits on...,2017-12-29,"['monday', 'pentagon']"
20290,fbi russia probe helped by australian diplomat...,2017-12-30,"['fbi russia', 'australian', 'nyt']"
20291,senior u s republican senator let mr mueller d...,2017-12-31,"['republican', 'mueller']"


In [ ]:
col_list = ['title','date','ALL']
fakedata=pd.read_csv("/content/drive/MyDrive/NLP_News/Data/Fake_P.csv",skipinitialspace=True,usecols=col_list)

In [ ]:
fakedata.columns = ['text','date','ALL']
fakedata.head()

,text,date,ALL
0,benghazi panel calls hillary to testify under ...,2015-03-31 00:00:00,"['hillary', 'white house', 'benghazi panel']"
1,oh no guess who funded the shrine to ted kennedy,2015-03-31 00:00:00,['ted kennedy']
2,apple s ceo says religious freedom laws are da...,2015-03-31 00:00:00,['apple s ceo']
3,flashback king obama commutes sentences of dru...,2015-03-31 00:00:00,['obama']
4,non violence hasn t worked reverend sam mostel...,2015-04-01 00:00:00,"['sam mosteller', 'mlk']"


In [ ]:
def check_similarity(test_data,total_data,sim_rate):
  cv = TfidfVectorizer(stop_words="english")
  docs_total=total_data["text"].tolist()
  docs_test=test_data["text"].tolist()
  cv.fit(docs_total+docs_test)

  total_tweets=cv.transform(docs_total)
  test_tweets=cv.transform(docs_test)
  
  cosine_similarities_temp = np.dot(test_tweets[0],total_tweets.T).T
  cosine_similarities = np.reshape(cosine_similarities_temp.toarray(),(-1,))
  # print(cosine_similarities)
  most_similar=np.where(cosine_similarities>sim_rate)
  list_sim=[]
  for i in list(most_similar[0]):
    list_sim.append(int(total_data.ind[i:i+1]))
  return list_sim

In [ ]:
def find_week(sample_time,total_time):
  x=[]
  for i in range(len(total_time)):
    x.append(np.abs((total_time[i]-sample_time).days))
  return np.argmin(np.array(x))

In [ ]:
m_list.shape

(18, 7)

In [ ]:
m_list

,Unnamed: 0,week,middle_time,model,tf_idf,start_ind,end_ind
0,0,1.0,2016-04-10 01:01:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,50594,52808
1,1,1.5,2016-04-12 01:01:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,51481,53879
2,2,2.0,2016-04-15 01:02:53,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,52809,55048
3,3,2.5,2016-04-17 01:02:53,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,53883,56061
4,4,3.0,2016-04-20 01:12:09,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,55049,57623
5,5,3.5,2016-04-22 01:12:09,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,56067,58209
6,6,4.0,2016-04-25 01:13:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,57624,59823
7,7,4.5,2016-04-27 01:13:55,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,58212,60820
8,8,5.0,2016-04-30 01:19:00,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,59824,61945
9,9,5.5,2016-05-02 01:19:00,b'\x80\x03csklearn.cluster._kmeans\nKMeans\nq\...,b'\x80\x03csklearn.feature_extraction.text\nTf...,60822,63006


In [ ]:
truedata=truedata[1230:1833]

In [ ]:
fakedata=fakedata[3378:4412]

In [ ]:
fakedata= fakedata.reset_index(drop=True)
fakedata

,text,date,ALL
0,bernie sanders will visit the vatican to speak...,2016-04-08 00:00:00,"['vatican', 'bernie sanders']"
1,white catholic schoolkids threaten to lynch bl...,2016-04-08 00:00:00,"['white catholic', 'schoolkids', 'lynch black']"
2,late night hosts out of new york city utterly ...,2016-04-08 00:00:00,"['late night', 'ted cruz', 'new york city']"
3,seth macfarlane wants his fellow bernie suppor...,2016-04-08 00:00:00,"['seth macfarlane', 'bernie']"
4,watch donald trump receives endorsement from n...,2016-04-08 00:00:00,"['neo nazis', 'donald trump']"
...,...,...,...
1029,bigot christians make a hilariously awful anti...,2016-05-24 00:00:00,"['christians', 'trans']"
1030,young republicans chairwoman resigns over trum...,2016-05-24 00:00:00,"['resigns', 'republicans']"
1031,breaking doj to seek death penalty for charles...,2016-05-24 00:00:00,"['doj', 'charleston']"
1032,republicans sabotage obamacare so they can cla...,2016-05-24 00:00:00,"['obamacare', 'republicans']"


In [ ]:
truedata= truedata.reset_index(drop=True)
truedata

,text,date,ALL
0,trump adviser says republicans won t have cont...,2016-04-08,['republicans']
1,papal official denies report sanders invited h...,2016-04-08,"['vatican', 'sanders']"
2,new jersey s christie loses bid to freeze atla...,2016-04-08,"['christie', 'atlantic city', 'new jersey']"
3,poll trump extends his lead among republicans ...,2016-04-08,['republicans']
4,obama to discuss islamic state at cia next week,2016-04-08,"['cia', 'islamic', 'next week', 'obama']"
...,...,...,...
598,vietnamese dissident says prevented by police ...,2016-05-24,"['obama', 'vietnamese']"
599,u s vietnam agree to allow peace corps into vi...,2016-05-24,"['first', 'peace corps', 'u s', 'vietnam']"
600,democratic presidential candidate sanders seek...,2016-05-24,"['kentucky', 'sanders', 'democratic']"
601,clinton turns down fox news offer to debate sa...,2016-05-24,"['clinton', 'sanders', 'fox news']"


In [ ]:
for i in range(fakedata.shape[0]):
  fakedata.date[i]=datetime.strptime(fakedata.date[i], "%Y-%m-%d %H:%M:%S")
for i in range(truedata.shape[0]):
  truedata.date[i]=datetime.strptime(truedata.date[i], "%Y-%m-%d")

In [ ]:
sim_ind=[]
all_ind=[]
for i in range(truedata.shape[0]):
  print(i)

  a=truedata.date[i]
  b=m_list.middle_time
  ind=find_week(a,np.array(b))

  model_idf=pickle.loads(eval(m_list.tf_idf[ind]))
  model_km=pickle.loads(eval(m_list.model[ind]))

  true_tweets=model_idf.transform([truedata.text[i]])
  true_label=model_km.predict(true_tweets)
  
  total_tweets=model_idf.transform( total.text[ m_list.start_ind[ind] : m_list.end_ind[ind] ])
  total_label=np.array(model_km.predict(total_tweets))

  mask=total_label==true_label[0]
  d_total=total[ m_list.start_ind[ind] : m_list.end_ind[ind]]
  d_total=d_total[mask]
  all_ind.append(d_total.index.tolist())
  if d_total.shape[0]>0:
    f=check_similarity(truedata[i:i+1],d_total,0.31)

    if len(f)>0:
      sim_ind.append(f)
      # print("kk")
    else:
      # print("ooo")
      sim_ind.append(-1)
  else:
    print("errrrroooooorrrr")
    sim_ind.append(-1)

  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
len(sim_ind)

603

In [ ]:
truedata["similar_ind"]=sim_ind
truedata["all_ind"]=all_ind
truedata[:50]

,text,date,ALL,similar_ind,all_ind
0,trump adviser says republicans won t have cont...,2016-04-08 00:00:00,['republicans'],-1,[52401]
1,papal official denies report sanders invited h...,2016-04-08 00:00:00,"['vatican', 'sanders']",-1,[51057]
2,new jersey s christie loses bid to freeze atla...,2016-04-08 00:00:00,"['christie', 'atlantic city', 'new jersey']",-1,[51765]
3,poll trump extends his lead among republicans ...,2016-04-08 00:00:00,['republicans'],-1,[52401]
4,obama to discuss islamic state at cia next week,2016-04-08 00:00:00,"['cia', 'islamic', 'next week', 'obama']",-1,"[51818, 52229, 52562]"
5,colorado lawmaker aims to outlaw pot laced gum...,2016-04-08 00:00:00,['colorado'],-1,[52537]
6,u s hands over pages of benghazi records house...,2016-04-08 00:00:00,"['u s hands', 'benghazi records house']",-1,[51996]
7,vatican trip could boost sanders fight against...,2016-04-08 00:00:00,"['sanders', 'clinton', 'vatican trip']",-1,[51057]
8,leak of senate encryption bill prompts swift b...,2016-04-08 00:00:00,['senate'],-1,"[52145, 52803]"
9,democrat sanders big fan of pope to attend vat...,2016-04-08 00:00:00,"['vatican conference', 'democrat', 'sanders']",-1,[51057]


In [ ]:
print(total.text[252483])
# # print(total.text[10234])
# # print(total.text[10565])
print(truedata.text[28])
# cv = TfidfVectorizer(stop_words="english")
# docs_total=total.text[33580:33581].tolist()
# docs_test=truedata.text[47:48].tolist()
# cv.fit(docs_total+docs_test)

# total_tweets=cv.transform(docs_total)
# test_tweets=cv.transform(docs_test)
  
# cosine_similarities_temp = np.dot(test_tweets[0],total_tweets.T).T
# cosine_similarities = np.reshape(cosine_similarities_temp.toarray(),(-1,))
# cosine_similarities

update gunman kills two staff at private school in saudi arabian capital security source riyadh
path cleared for congress to consider u s arms sale to riyadh state department


In [ ]:
truedata.to_csv("/content/drive/MyDrive/NLP_News/Test_data/true7.csv")

In [ ]:
sim_ind=[]
all_ind=[]
for i in range(fakedata.shape[0]):
  print(i)

  a=fakedata.date[i]
  b=m_list.middle_time
  ind=find_week(a,np.array(b))

  model_idf=pickle.loads(eval(m_list.tf_idf[ind]))
  model_km=pickle.loads(eval(m_list.model[ind]))

  true_tweets=model_idf.transform([fakedata.text[i]])
  true_label=model_km.predict(true_tweets)
  
  total_tweets=model_idf.transform( total.text[ m_list.start_ind[ind] : m_list.end_ind[ind] ])
  total_label=np.array(model_km.predict(total_tweets))

  mask=total_label==true_label[0]
  d_total=total[ m_list.start_ind[ind] : m_list.end_ind[ind]]
  d_total=d_total[mask]
  all_ind.append(d_total.index.tolist())
  if d_total.shape[0]>0:
    f=check_similarity(fakedata[i:i+1],d_total,0.21)
    if len(f)>0:
      sim_ind.append(f)
      # print("kk")
    else:
      # print("ooo")
      sim_ind.append(-1)
  else:
    print("erroooorrr")
    sim_ind.append(-1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
fakedata["similar_ind"]=sim_ind
fakedata["all_ind"]=all_ind
# fakedata[fakedata.similar_ind!=-1]

In [ ]:
fakedata[fakedata.similar_ind!=-1]

,text,date,ALL,similar_ind,all_ind
2,late night hosts out of new york city utterly ...,2016-04-08 00:00:00,"['late night', 'ted cruz', 'new york city']",[51107],"[51107, 52496]"
3,seth macfarlane wants his fellow bernie suppor...,2016-04-08 00:00:00,"['seth macfarlane', 'bernie']",[52455],"[50600, 50618, 50619, 50626, 50629, 50630, 506..."
15,watch nj man may face jail for flying donald t...,2016-04-08 00:00:00,"['donald trump flag', 'nj']",[50976],[50976]
17,n c naacp plans sit ins to force repeal of ant...,2016-04-08 00:00:00,['n c naacp'],[52749],"[50600, 50618, 50619, 50626, 50629, 50630, 506..."
23,bruce springsteen just unleashed his fury on n...,2016-04-08 00:00:00,"['north carolina', 'bruce springsteen']","[51006, 51010, 51027, 51029]","[51006, 51010, 51027, 51029]"
...,...,...,...,...,...
995,trump lamborghini rolls up to white house give...,2016-05-22 00:00:00,"['lamborghini', 'white house']",[70079],[70079]
998,breaking fbi investigating clinton crony virgi...,2016-05-23 00:00:00,"['clinton', 'fbi', 'terry mcauliffe', 'virginia']",[71326],"[71325, 71326]"
1002,planned parenthood evacuated in florida hospit...,2016-05-23 00:00:00,['florida'],[71252],"[71252, 71932]"
1025,how john oliver and a cat created an internati...,2016-05-24 00:00:00,['john oliver'],"[70901, 71238, 71303]","[69841, 69842, 69844, 69852, 69857, 69860, 698..."


In [ ]:
# # print(fakedata.text[23])
# print(total.text[4044])
# cv = TfidfVectorizer(stop_words="english")
# docs_total=total.text[4044:4045].tolist()
# docs_test=fakedata.text[23:24].tolist()
# cv.fit(docs_total+docs_test)

# total_tweets=cv.transform(docs_total)
# test_tweets=cv.transform(docs_test)
  
# cosine_similarities_temp = np.dot(test_tweets[0],total_tweets.T).T
# cosine_similarities = np.reshape(cosine_similarities_temp.toarray(),(-1,))
# cosine_similarities

In [ ]:
fakedata.to_csv("/content/drive/MyDrive/NLP_News/Test_data/fake7.csv")